In [ ]:
#
import os
import json
import numpy as np
import pandas as pd
from itertools import accumulate, permutations, combinations, product
from sklearn.cluster import DBSCAN

from IPython.display import display
from IPython.display import Markdown
from tabulate import tabulate


## Introduction


The Guardians, a lost, enigmatic civilization that left us beacons and ruins, millions of years old, yet all still functional. They were technologically advanced and colonized part of the Orion arm long before humans learned to travel between stars, long before we even existed.

Not one decade after their initial discovery a remarkable 300 systems have been discovered with remains of this mysterious civilization.



For information on Guardians

[Canonn Research / The Guardians](https://canonn.science/codex/the-guardians/)



### An observation

In [ ]:
raw_guardian_clustermarkers="https://raw.githubusercontent.com/fenke/terzijde/main/posts/ed-guardians-1/guardian-clusters.json"
ed_astro_galmap_custom_url="https://edastro.com/galmap/?custom="
def embed_astro(markers):
    if isinstance(markers, list):
        source = f"{ed_astro_galmap_custom_url}{';'.join([str(s) for s in markers])}"
    else:
        source = f"{ed_astro_galmap_custom_url}{str(markers)}"
        
    return f'''
<iframe width=100% height=640 src="{source}"></iframe> 
'''

The [Interactive Galactic Map](https://edastro.com/galmap/) with EDAstro can display markersfor known systems with guardian sites, the result can be seen below. (reload page if the image remains black after the cookie thing)

In [ ]:
Markdown(f"""

{embed_astro(raw_guardian_clustermarkers)}


"""
)





<iframe width=100% height=640 src="https://edastro.com/galmap/?custom=https://raw.githubusercontent.com/fenke/terzijde/main/posts/ed-guardians-1/guardian-clusters.json"></iframe> 





Looking at this map we immediately recognize these markers appear to lie on two lines that intersect somewhere at the border between the Formidine Rift and the Errant Marches.

### A first look

Canonn Research maintains data on discoveries related to the Guardians. We use the coordinate data on Guardian Sites, Ruins and Beacons from Canonn Research.

In [ ]:
# Terzijde/data/guardian/Canonn - Guardians - Brain Tree Sites.csv
# Terzijde/terzijde/posts/ed-guardians-1/index.ipynb
guardiandata_path = os.path.join(os.getcwd(), '..', '..', 'data', 'guardian')

guardiandata_files = {n.split(' - ')[2].split('.')[-2]:os.path.join(guardiandata_path, n) for n in os.listdir(guardiandata_path) if 'Canonn - Guardians' in n}
guardiandata = {n:pd.read_csv(p) for n,p in guardiandata_files.items()}

In [ ]:
soi = ['Ruins','Structures'] + ['Beacons']

# intersection and unions of columnnames
inter_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 & D2)][-1])
union_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 | D2)][-1])

# order them like in the Beacon's file
column_order_inter = {c:i for c,i in zip(guardiandata['Guardian Beacons'].columns, range(len(guardiandata['Guardian Beacons'].columns))) }
inter_columns = sorted(inter_columns, key=lambda I:column_order_inter.get(I,100))
union_columns = sorted(union_columns, key=lambda I:column_order_inter.get(I,100))

In [ ]:
# drop of beacon markers
for s in soi:
    continue
    with open(f"guardian-{s.lower()}.json", 'wt') as of:
        json.dump(dict(markers=[
                dict(
                    pin='cyan',
                    text=str(row),
                    **{c:v for c,v in zip(['x','y','z'], row[['x','y','z']])}
                )
                for index,row in guardiandata['Guardian '+s].iterrows()
            ]), of, indent=3)

In [ ]:
#[row for index,row in guardiandata['Guardian Beacons'].iterrows()]

In [ ]:
soi_systemnames = np.asarray(
    [[g]+[r[c] for c in inter_columns] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()] )
soi_coordinates = np.asarray([[r[c] for c in ['x', 'y', 'z']] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()])
#print(soi_coordinates.shape, soi_systemnames.shape)

In [ ]:
soi_systems = {r['System Name']:np.asarray([r[c] for c in ['x', 'y', 'z']]) for g in soi for i, r in guardiandata['Guardian '+g].iterrows()}
soi_systemnames = np.asarray(
    [[g]+[r[c] for c in inter_columns] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()] )
soi_coordinates = np.asarray([[r[c] for c in ['x', 'y', 'z']] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()])
#print(soi_coordinates.shape, soi_systemnames.shape)

In [ ]:
display(Markdown(f""" From the {len(guardiandata)} files in the dataset we will be looking at the {len(soi)} files 
with finds on {', '.join(soi[0:-1])} and {soi[-1:][0]} totalling {len(soi_systems)} systems.
""" ))

 From the 10 files in the dataset we will be looking at the 3 files 
with finds on Ruins, Structures and Beacons totalling 344 systems.


In [1]:
coord_clusters = DBSCAN(eps=200, min_samples=3).fit(soi_coordinates)
#print(np.unique(coord_clusters.labels_))
#print(np.count_nonzero(np.less(coord_clusters.labels_,0)))
#print([(l, np.count_nonzero(np.equal(coord_clusters.labels_,l))) for l in np.unique(coord_clusters.labels_)])


NameError: name 'DBSCAN' is not defined

In [ ]:
outliers = np.copy(soi_coordinates[np.less(coord_clusters.labels_,0)])
outlier_systemnames = np.copy(soi_systemnames[np.less(coord_clusters.labels_,0)])
outlier_systemnames[:,2].tolist()

['SYNUEFE XR-H D11-102',
 'TRAPEZIUM SECTOR YU-X C1-2',
 'TRAPEZIUM SECTOR YU-X C1-2',
 'WREGOE BU-Y B2-0',
 'WREGOE BU-Y B2-0']

In [ ]:

with open(f"guardian-clusters.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='cyan',
                text=f'Center of cluster {row[0]} with {row[2]guardian_outliers_json_pins} guardian sites',
                **{c:v for c,v in zip(['x','y','z'], row[1])}
            )
            for row in [
                (l, np.round(np.mean(soi_coordinates[np.equal(coord_clusters.labels_,l)], axis=0),2).tolist(), np.count_nonzero(np.equal(coord_clusters.labels_,l)) ) 
                for l in np.unique(coord_clusters.labels_) 
                if not l < 0]
        ]), of, indent=3)

In [ ]:

with open(f"guardian-outliers.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='red',
                text='\n'.join([f"{cn:20}: {v}" for cn, v in zip(['type'] + inter_columns,row[0])]),
                **{c:v for c,v in zip(['x','y','z'], row[1])}
            )
            for row in [(n,c) for n,c in zip(outlier_systemnames, outliers)]
        ] ), of, indent=3)